<a href="https://colab.research.google.com/github/gpandu/GAN/blob/main/GPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tokenization:



*   Subword Tokenization : Keep frequent words and break rearer words into subwords
*   A statastical Alogrothm learns how to do this based on corpus.

> Ex: Listeria ---> "list" , "eria"

> "list" is common word likely to be found in dictionary


*   Tokenization has better chance of handling OOV words while decreasing the size of the overall dictionary.   









In [2]:
!pip install transformers
!pip install tokenizers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00


In [3]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.BPE())

In [4]:
from datasets import load_dataset

# load code dataset
raw_dataset = load_dataset("code_search_net", "python")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [5]:
raw_dataset["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [6]:
def get_training_corpus():
    dataset = raw_dataset["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]


In [8]:
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)


In [9]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [12]:
tokenizer.num_special_tokens_to_add(is_pair = False)

0

In [15]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokenizer.enable_padding(length=256, pad_id = 0, pad_token = "<|endoftext|>")
encoding = tokenizer.encode(example)
print(encoding.ids)
print(encoding.attention_mask)

[295, 771, 63, 7245, 8, 65, 12, 271, 274, 232, 290, 2073, 256, 2030, 3948, 572, 65, 64, 350, 572, 66, 11957, 232, 302, 231, 382, 271, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [17]:
decode = tokenizer.decode(encoding.ids)

def Ġadd _ numbers ( a , Ġb ): ĊĠĠĠ Ġ""" Add Ġthe Ġtwo Ġnumbers Ġ` a ` Ġand Ġ` b `.""" ĊĠĠĠ Ġreturn Ġa Ġ+ Ġb


In [5]:
import math
import numpy as np
import tensorflow as tf

Multi Head Attention



*   Each Attention head performs Scaled Dot Product Self-Attention operation where given Keys, Query and Values, the return matrix of values given by below operation.

        Attention(Q,K,V) = softmax((Q*Transpose(K))/sqrt(d))*V





In [4]:
def scaled_dot_product_attention(query, key, value, mask=None):
  key_dims = tf.cast(tf.shape(key)[-1], tf.float32)
  scaled_scores = tf.matmul(query, key, transpose_b=True) / np.sqrt(key_dims)

  if mask is not None:
    scaled_scores = tf.where(mask==0, -np.inf, scaled_scores)

  softmax = tf.keras.layers.Softmax()
  weights = softmax(scaled_scores)
  return tf.matmul(weights, value), weights



Dimensions

In [ ]:
seq_len = 100
embed_dim = 512

queries = np.random.rand(seq_len, embed_dim)
keys = np.random.rand(seq_len, embed_dim)
values = np.random.rand(seq_len, embed_dim)

print("Queries:\n", queries.shape)

Queries:
 (100, 512)


In [ ]:
output, attn_weights = scaled_dot_product_attention(queries, keys, values)

print("Output\n", output, "\n")
print("Weights\n", attn_weights)

Output
 tf.Tensor(
[[0.5283472  0.47262007 0.48678786 ... 0.4829216  0.48645368 0.5032488 ]
 [0.5207001  0.47411957 0.4903704  ... 0.48768327 0.4847127  0.49883205]
 [0.5268495  0.47535077 0.48665828 ... 0.48832378 0.48390666 0.5004193 ]
 ...
 [0.51870614 0.47969958 0.49663618 ... 0.48446965 0.4867398  0.50444645]
 [0.52074957 0.4760783  0.4912476  ... 0.47999552 0.48633608 0.49695626]
 [0.52396476 0.4755557  0.49060395 ... 0.47889382 0.48861432 0.5024368 ]], shape=(100, 512), dtype=float32) 

Weights
 tf.Tensor(
[[0.00715606 0.01213571 0.00987343 ... 0.01058201 0.01125814 0.00906746]
 [0.00832826 0.01025666 0.00907701 ... 0.00845557 0.01180658 0.01047523]
 [0.00685807 0.01074812 0.00893654 ... 0.00815846 0.00981395 0.00853847]
 ...
 [0.0063051  0.01066695 0.00989037 ... 0.00842666 0.01008323 0.00987076]
 [0.0084636  0.01041168 0.0096035  ... 0.00817274 0.00860864 0.0100768 ]
 [0.00760248 0.01128288 0.00973016 ... 0.00983269 0.00964166 0.01040739]], shape=(100, 100), dtype=float32)


**Generating queries, keys, and values for multiple heads.**

> Now that we have a way to calculate self-attention, let's actually generate the input queries, keys, and values for multiple heads.

>  each attention head had its own separate set of query, key, and value weights. Each weight matrix was of dimension  d x d/h  where h was the number of heads.




In [ ]:
batch_size = 1000
seq_len = 100
embed_dim = 512
num_heads = 4
head_dim = embed_dim // num_heads

In [ ]:
def get_weight_matrix(x_dims, y_dims):
  return np.random.rand(x_dims, y_dims).round(1)


In [ ]:
query_heads = []
key_heads = []
value_heads = []
for i in range(0,num_heads):
  query_heads.append(get_weight_matrix(embed_dim, head_dim))
  key_heads.append(get_weight_matrix(embed_dim, head_dim))
  value_heads.append(get_weight_matrix(embed_dim, head_dim))


query_heads[0].shape

(512, 128)

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.d_model = d_model
    self.num_heads = num_heads

    self.d_head = self.d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(self.d_model, use_bias=False)
    self.wk = tf.keras.layers.Dense(self.d_model, use_bias=False)
    self.wv = tf.keras.layers.Dense(self.d_model, use_bias=False)

    # Linear layer to generate the final output.
    self.dense = tf.keras.layers.Dense(self.d_model)

  def split_heads(self, x):
    batch_size = x.shape[0]

    split_inputs = tf.reshape(x, (batch_size, -1, self.num_heads, self.d_head))
    return tf.transpose(split_inputs, perm=[0, 2, 1, 3])

  def merge_heads(self, x):
    batch_size = x.shape[0]

    merged_inputs = tf.transpose(x, perm=[0, 2, 1, 3])
    return tf.reshape(merged_inputs, (batch_size, -1, self.d_model))

  def call(self, q, k, v, mask):
    qs = self.wq(q)
    ks = self.wk(k)
    vs = self.wv(v)

    qs = self.split_heads(qs)
    ks = self.split_heads(ks)
    vs = self.split_heads(vs)

    output, attn_weights = scaled_dot_product_attention(qs, ks, vs, mask)
    output = self.merge_heads(output)

    return self.dense(output), attn_weights


In [ ]:
def feed_forward_network(d_model, hidden_dim):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(hidden_dim, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])

In [ ]:
class DecoderBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
    super(DecoderBlock, self).__init__()

    self.mhsa1 = MultiHeadAttention(d_model, num_heads)

    self.ffn = feed_forward_network(d_model, hidden_dim)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.layernorm1 = tf.keras.layers.LayerNormalization()
    self.layernorm2 = tf.keras.layers.LayerNormalization()

  # Note the decoder block takes two masks. One for the first MHSA, another
  # for the second MHSA.
  def call(self, target, training, decoder_mask):
    mhsa_output1, attn_weights = self.mhsa1(target, target, target, decoder_mask)
    mhsa_output1 = self.dropout1(mhsa_output1, training=training)
    mhsa_output1 = self.layernorm1(mhsa_output1 + target)

    ffn_output = self.ffn(mhsa_output1)
    ffn_output = self.dropout2(ffn_output, training=training)
    output = self.layernorm2(ffn_output + mhsa_output1)

    return output, attn_weights


In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_seq_len, dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.max_seq_len = max_seq_len

    self.token_embed = tf.keras.layers.Embedding(target_vocab_size, self.d_model)
    self.pos_embed = tf.keras.layers.Embedding(max_seq_len, self.d_model)

    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    self.blocks = [DecoderBlock(self.d_model, num_heads, hidden_dim, dropout_rate) for _ in range(num_blocks)]

  def call(self, target, training, decoder_mask):
    token_embeds = self.token_embed(target)

    # Generate position indices.
    num_pos = target.shape[0] * self.max_seq_len
    pos_idx = np.resize(np.arange(self.max_seq_len), num_pos)
    pos_idx = np.reshape(pos_idx, target.shape)

    pos_embeds = self.pos_embed(pos_idx)

    x = self.dropout(token_embeds + pos_embeds, training=training)

    for block in self.blocks:
      x, weights = block(x, training, decoder_mask)

    return x, weights

In [ ]:
def loss_func(targets, logits):
  ce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

  mask = tf.cast(tf.math.not_equal(targets, 0), tf.float32)

  return ce_loss(targets, logits, sample_weight=mask)

In [ ]:
class GPT2Model(tf.keras.Model):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_input_len, max_target_len, dropout_rate=0.1):
    super(GPT2Model, self).__init__()

    self.decoder = Decoder(num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
                           max_target_len, dropout_rate)

    # The final dense layer to generate logits from the decoder output.
    self.output_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, input_seqs, training, decoder_mask):

    decoder_output, decoder_attn_weights = self.decoder(input_seqs, training, decoder_mask)

    return self.output_layer(decoder_output), decoder_attn_weights
